### Hate and Toxic Comment Detection 

**DataLab USAL - Project 2019/2020** 

### 01. Corpus Exploration

Este primer Notebook servirá como primera toma de contacto. Cargaremos los tweets y daremos el formato adecuado a la información de la que disponemos. Además, realizaremos un análisis exploratorio básico sobre las variables con algunos gráficos descriptivos.

---

#### Librerías

In [2]:
# standard
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# misc
import gzip
import logging
import codecs
global lcode
import warnings
warnings.filterwarnings("ignore")

# nlp
import re
from collections import Counter
from nltk import word_tokenize
import nltk
import spacy
import itertools
import collections
from nltk.corpus import stopwords
import networkx

# pandas parameters
pd.set_option('display.max_columns', None)

# seaborn parameters
sns.set(font_scale=1.5)
sns.set_style("whitegrid")

%matplotlib inline

### Lectura de tweets

In [34]:
col_names = ["col1", "date", "user", "tweet", "via", "id", "followers", "following",
             "statuses", "loc", "link", "col12", "col13", "user_bio", "col15", "col16",
             "col17", "tweet_type", "col19", "politician_mention", "col21", "politician_rt", "col23", "col24",
             "hashtag", "col26","count_creation", "col28", "col29", "col30"]

In [35]:
files = os.listdir('./data/')

In [40]:
tweets_dataframe = pd.DataFrame()

for file_name in files:
    tweets = pd.read_csv("./data/" + file_name, sep="\t", names=col_names, encoding="latin-1")
    tweets_dataframe = pd.concat([tweets_dataframe, tweets])

In [41]:
tweets_dataframe

,col1,date,user,tweet,via,id,followers,following,statuses,loc,link,col12,col13,user_bio,col15,col16,col17,tweet_type,col19,politician_mention,col21,politician_rt,col23,col24,hashtag,col26,count_creation,col28,col29,col30
0,1147190909345456130,2019-07-05 17:10:01,@mikemit87350572,@InesArrimadas Si quieres los votos de VOX vas...,via=Twitter for Android,id=1073287763150155776,followers=22,following=50,statuses=1083,loc=None,None,None,mikemitch,None,None,None,None,reply,1146861516261646336,InesArrimadas,None,None,None,None,None,es,2018-12-13 18:45:18,False,https://pbs.twimg.com/profile_images/107328817...,https://twitter.com/mikemit87350572/status/114...
1,1147190998344327168,2019-07-05 17:10:22,@DulantziNabar,RT @Irene_Montero_: En Unidas Podemos todos so...,via=Twitter for Android,id=873599918878973952,followers=2028,following=1911,statuses=17637,loc=Bilbao,None,None,Dulantziâ¨ð,No se llega al alba sino por el sendero de la ...,http://pbs.twimg.com/amplify_video_thumb/11471...,video,None,RT,None,None,1147135545413439488,Irene_Montero_,None,None,None,es,2017-06-10 17:57:06,False,https://pbs.twimg.com/profile_images/114652331...,https://twitter.com/DulantziNabar/status/11471...
2,1147190999225122817,2019-07-05 17:10:22,@CdpdDavid,RT @InesArrimadas: SÃ¡nchez ya ha elegido a su...,via=Twitter for Android,id=3248003764,followers=1354,following=1498,statuses=66024,loc=None,https://www.elespanol.com/opinion/tribunas/201...,None,David Santos,Cineasta Guionista y escritor. Apasionado d la...,None,None,None,RT,None,None,1147137586550493186,InesArrimadas,None,None,None,es,2015-05-12 14:24:08,False,https://pbs.twimg.com/profile_images/746432913...,https://twitter.com/CdpdDavid/status/114719099...
3,1147191001280372736,2019-07-05 17:10:23,@Reinaraf1962,RT @Irene_Montero_: En Unidas Podemos todos so...,via=Twitter for iPhone,id=2975012789,followers=29,following=65,statuses=7757,loc=None,None,None,Rafael Reina,None,http://pbs.twimg.com/amplify_video_thumb/11471...,video,None,RT,None,None,1147135545413439488,Irene_Montero_,None,None,None,es,2015-01-12 17:35:16,False,https://pbs.twimg.com/profile_images/555795320...,https://twitter.com/Reinaraf1962/status/114719...
4,1147191021652066307,2019-07-05 17:10:27,@juannquillo,@sanchezcastejon Lo que EspaÃ±a necesita es de...,via=Twitter for Android,id=301353218,followers=50,following=171,statuses=3416,loc=None,None,None,Juan C,None,None,None,None,reply,1146877439764848651,sanchezcastejon,None,None,None,None,None,es,2011-05-19 09:58:20,False,https://pbs.twimg.com/profile_images/560476623...,https://twitter.com/juannquillo/status/1147191...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121949,1172247412120965120,2019-09-12 20:35:37,@AraceliOtero60,"RT @Pablo_Iglesias_: El Estado de California, ...",via=Twitter for Android,id=615536207,followers=340,following=518,statuses=11271,loc=EspaÃ±a,None,None,Araceli,"Defiendo los Derechos Humanos, La Sanidad y La...",http://pbs.twimg.com/media/EEQb4WHXUAASpPH.jpg,photo,None,RT,None,None,1172112620675055616,Pablo_Iglesias_,None,None,None,es,2012-06-22 21:19:45,False,https://pbs.twimg.com/profile_images/117146067...,https://twitter.com/AraceliOtero60/status/1172...
121950,1172247415207993345,2019-09-12 20:35:37,@JulioGD77,RT @Santi_ABASCAL: Una cÃ©lula yihadista plane...,via=Twitter Web App,id=1216660598,followers=942,following=1463,statuses=5566,loc=None,https://okdiario.com/sociedad/yihadistas-deten...,None,Julio Guerrero,Promover la excelencia. Patriotismo. ðªð¸...,None,None,None,RT,None,None,1172180948575817736,Santi_ABASCAL,None,None,None,es,2013-02-24 21:59:48,False,https://pbs.twimg.com/profile_images/997183065...,https://twitter.com/JulioGD77/status/117224741...
121951,1172247421407178752,2019-09-12 20:35:39,@mariajo_mami,RT @Santi_ABASCAL: Una cÃ©lula yihadista plane...,via=Twitter for Android,id=169861144,followers=70,following=89,statuses=5648,loc=None,https://okdiario.

### Eliminar variables inservibles y renombrar

In [14]:
tweets.drop(["col1", "id", "statuses", "col13", "col15", "col16", "col19", 
             "col21", "col23", "col26", "col29", "col30"], axis=1, inplace=True)

In [15]:
tweets.rename(columns={"col2": "date",
                       "col3": "user",
                       "col11": "link",
                       "col14": "user_bio",
                       "col18": "tweet_type",
                       "col20": "politician_mention",
                       "col22": "politician_rt",
                       "col25": "hashtag",
                       "col27": "count_creation"}, inplace=True)

In [16]:
tweets.head(20)

,date,user,tweet,via,followers,following,loc,link,col12,user_bio,col17,tweet_type,politician_mention,politician_rt,col24,hashtag,count_creation,col28
0,2019-07-05 17:10:01,@mikemit87350572,@InesArrimadas Si quieres los votos de VOX vas...,via=Twitter for Android,followers=22,following=50,loc=None,None,None,None,None,reply,InesArrimadas,None,None,None,2018-12-13 18:45:18,False
1,2019-07-05 17:10:22,@DulantziNabar,RT @Irene_Montero_: En Unidas Podemos todos so...,via=Twitter for Android,followers=2028,following=1911,loc=Bilbao,None,None,No se llega al alba sino por el sendero de la ...,None,RT,None,Irene_Montero_,None,None,2017-06-10 17:57:06,False
2,2019-07-05 17:10:22,@CdpdDavid,RT @InesArrimadas: Sánchez ya ha elegido a sus...,via=Twitter for Android,followers=1354,following=1498,loc=None,https://www.elespanol.com/opinion/tribunas/201...,None,Cineasta Guionista y escritor. Apasionado d la...,None,RT,None,InesArrimadas,None,None,2015-05-12 14:24:08,False
3,2019-07-05 17:10:23,@Reinaraf1962,RT @Irene_Montero_: En Unidas Podemos todos so...,via=Twitter for iPhone,followers=29,following=65,loc=None,None,None,None,None,RT,None,Irene_Montero_,None,None,2015-01-12 17:35:16,False
4,2019-07-05 17:10:27,@juannquillo,@sanchezcastejon Lo que España necesita es de ...,via=Twitter for Android,followers=50,following=171,loc=None,None,None,None,None,reply,sanchezcastejon,None,None,None,2011-05-19 09:58:20,False
5,2019-07-05 17:10:29,@St2Cecilia,@InesArrimadas Tu cuenta cada vez parece mas a...,via=Twitter for Android,followers=904,following=1162,loc=None,None,None,Aut viam inveniam aut faciam,None,reply,InesArrimadas,None,None,None,2019-04-28 21:06:30,False
6,2019-07-05 17:10:35,@elivarfe,"@Irene_Montero_ Excepto de caja ingenieros, ah...",via=Twitter for iPhone,followers=108,following=145,loc=None,None,None,None,None,reply,Irene_Montero_,None,None,None,2011-10-24 16:20:56,False
7,2019-07-05 17:10:35,@Cazafalacias1,@InesArrimadas Por culpa de vox o por culpa de...,via=Twitter for Android,followers=217,following=1053,loc=espein,None,None,"Me gusta discutir, mucho. Pero no de cualquier...",None,reply,InesArrimadas,None,None,None,2019-04-09 01:30:07,False
8,2019-07-05 17:10:37,@cejuemme,RT @InesArrimadas: Ayer VOX votó que no a un G...,via=Twitter for iPhone,followers=846,following=4775,loc=@Germany @Ruesselsheim @Rodgau @USA @Calif...,None,None,@RiverPlate @BuenosAires 🇦🇷@operamusica @_Rich...,None,RT,None,InesArrimadas,None,None,2017-08-01 07:57:56,False
9,2019-07-05 17:10:41,@joserleo,"@sanchezcastejon @PSOE ""no puede quedarse espe...",via=Twitter for Android,followers=211,following=284,loc=None,None,None,"Libertad, igualdad, fraternidad. ❤💛💜",Uy. https://t.co/F8aVd3kBMj,quote,None,None,pvallin,None,2010-08-15 16:19:46,False


### Limpieza de variables informativas

In [17]:
tweets["via"] = tweets["via"].str.replace("via=", "")
tweets["followers"] = tweets["followers"].str.replace("followers=", "")
tweets["following"] = tweets["following"].str.replace("following=", "")
tweets["loc"] = tweets["loc"].str.replace("loc=", "")

### Formato de variables

In [18]:
tweets["date"] = pd.to_datetime(tweets["date"], format="%Y-%m-%d %H:%M:%S", errors="coerce")
tweets["date"].dtype

dtype('<M8[ns]')

In [19]:
tweets["followers"] = pd.to_numeric(tweets["followers"], errors="coerce")
tweets["followers"].dtype

dtype('float64')

In [20]:
tweets["following"] = pd.to_numeric(tweets["following"], errors="coerce")
tweets["following"].dtype

dtype('float64')

### Limpieza de tweets

In [21]:
tweets["tweet"] = tweets["tweet"].astype(str)

In [22]:
politicians = ["@InesArrimadas", "@Irene_Montero_", "@carmencalvo_", "@cayetanaAT", 
               "@Albert_Rivera", "@Pablo_Iglesias_", "@sanchezcastejon", "@pablocasado_",
               "@Santi_ABASCAL", "@monasterioR"]

In [23]:
def tweet_cleaning(df):
    for polit in politicians:
        df = df.apply(lambda elem: " ".join(re.sub(polit, "", elem).split()))
        
    return df

In [24]:
def tweet_cleaning2(df):
    df = df.str.lower()
    df = df.apply(lambda elem: " ".join(re.sub("á", "a", elem).split()))
    df = df.apply(lambda elem: " ".join(re.sub("é", "e", elem).split()))
    df = df.apply(lambda elem: " ".join(re.sub("í", "i", elem).split()))
    df = df.apply(lambda elem: " ".join(re.sub("ó", "o", elem).split()))
    df = df.apply(lambda elem: " ".join(re.sub("ú", "u", elem).split()))
    df = df.apply(lambda elem: " ".join(re.sub("ñ", "n", elem).split()))
    df = df.apply(lambda elem: " ".join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|http.+?|^rt", " ", elem).split()))
    return df

In [52]:
def clean_text(text):
    regex_dict = { r'á' : 'a', 'é' : 'e', 'í' : 'i', 'ó' : 'o', 'ú' : 'u' , '[^\w#]' : ' ', '[ 0-9 ]' : ''}
    # Create a regular expression  from the dictionary keys
    if isinstance(text, str):
        regex = re.compile(r'(%s)' % "|".join(regex_dict.keys()))
        lst = regex.sub(lambda mo: regex_dict[[ k for k in regex_dict if re.search(k, mo.string[mo.start():mo.end()])][0]], text).lower()
    else:
        lst = ' '
    return lst

In [53]:
tweets_dataframe.loc[:,'tweet'] = tweets_dataframe.loc[:,'tweet'].apply(clean_text)

In [25]:
tweets['tweet'] = tweet_cleaning(tweets['tweet'])

In [ ]:
tweets['tweet'] = tweet_cleaning2(tweets['tweet'])

In [ ]:
tweets["tweet"]

#### Información del dataset

In [ ]:
print("Número de tweets recogidos: " + str(tweets.shape[0]))
print("Variables informativas/tweet: " + str(tweets.shape[1]))

## Stop Words

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words_set = set(stopwords.words("spanish"))

In [ ]:
stop_words = list(stop_words_set)
stop_words

In [ ]:
def stop_cleaning(stop_word_list):
    words_cleaned = [w.replace('á', 'a') for w in stop_word_list]
    words_cleaned = [w.replace('é', 'e') for w in words_cleaned]
    words_cleaned = [w.replace('í', 'i') for w in words_cleaned]
    words_cleaned = [w.replace('ó', 'o') for w in words_cleaned]
    words_cleaned = [w.replace('ú', 'u') for w in words_cleaned]
    return words_cleaned

In [ ]:
stopss = stop_cleaning(stop_words)
stopss

### Recuento palabras más habituales

Función para tokenizar, eliminar stop words y hacer recuento de palabras más  habituales

In [ ]:
def remove_stopwords(words):
    new_words = []
    for word in words:
        if word not in stopss:
            new_words.append(word)
    return new_words

In [ ]:
def num_palabras(df):
    tokenized_tweet = df['tweet'].apply(lambda x: x.split())

    tokenized_tweet = remove_stopwords(tokenized_tweet)

    all_words_new = list(itertools.chain(tokenized_tweet)) # tokenized_tweet

    counts_new = collections.Counter(all_words_new)

    count = counts_new.most_common(20)

    return count

In [ ]:
count = num_palabras(tweets)

In [ ]:
def grafico(count):
    clean_tweets_nsw = pd.DataFrame(count, columns=['words', 'count'])

    fig, ax = plt.subplots(figsize=(8, 8))

    # Plot horizontal bar graph
    clean_tweets_nsw.sort_values(by='count').plot.barh(x='words', y='count',
                                                       ax=ax, color="purple")

    ax.set_title("Common words found (without stop words)")

    return plt.show()

In [ ]:
grafico(count)

### Seleccionamos los retweets y vemos los # mas habituales

In [ ]:
tweets_RT = tweets.loc[tweets["tweet_type"] == "RT"]

In [ ]:
counts = collections.Counter(tweets_RT["hashtag"])
count = counts.most_common(10)
count

In [ ]:
tweets['date'] = pd.to_datetime(tweets['date'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
tweets.head(5)

In [ ]:
tweets['date'] = pd.to_datetime(tweets['date']).dt.floor('d')
tweets.head(5)

In [ ]:
tweets_by_date = tweets['date'].value_counts().rename_axis('date').reset_index(name='count')
tweets_by_date = tweets_by_date.sort_values(by=['date'], )
tweets_by_date

In [ ]:
fig = plt.figure()
plt.plot(tweets_by_date['date'],tweets_by_date['count'])
plt.xlabel('DATES')
plt.ylabel('TWEETS COUNT')
plt.xticks(rotation=45)

In [ ]:
%%HTML
<img style="float: right;" src="datalab.png" width="400px"/>